### Instalando bibliotecas

In [2]:
%%capture
pip install selenium

In [3]:
%%capture
pip install xlrd

In [4]:
%%capture
pip install geopy

In [6]:
import time
import json
import os
import unicodedata
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

# ==========================================
# CONFIGURAÇÃO DE BUSCA (INSIRA AQUI)
# ==========================================
CIDADE = "Conde"
ESTADO = "Paraiba"
CHECK_IN = "2026-03-10"  # Formato: AAAA-MM-DD
CHECK_OUT = "2026-03-15" # Formato: AAAA-MM-DD
# ==========================================

def format_location_for_url(cidade, estado):
    """Formata cidade e estado para o padrão do Airbnb (ex: Conde--Paraiba)."""
    def normalize(text):
        # Remove acentos e substitui espaços por hífens
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
        return text.strip().replace(" ", "-")
    
    cidade_fmt = normalize(cidade)
    estado_fmt = normalize(estado)
    return f"{cidade_fmt}--{estado_fmt}"

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def inject_dates_to_url(url, checkin, checkout):
    """Adiciona parâmetros de data à URL do Airbnb."""
    u = urlparse(url)
    query = parse_qs(u.query)
    query['check_in'] = [checkin]
    query['check_out'] = [checkout]
    new_query = urlencode(query, doseq=True)
    return urlunparse((u.scheme, u.netloc, u.path, u.params, new_query, u.fragment))

def scrape_airbnb_links():
    # Formatar a localização e construir a URL base
    loc_path = format_location_for_url(CIDADE, ESTADO)
    base_url = f"https://www.airbnb.com.br/s/{loc_path}/homes"
    
    # Injetar datas na URL
    target_url = inject_dates_to_url(base_url, CHECK_IN, CHECK_OUT)
    print(f"Iniciando busca para: {CIDADE}, {ESTADO}")
    print(f"URL de busca: {target_url}")
    
    driver = setup_driver()
    all_links = set()
    page_number = 1
    
    try:
        driver.get(target_url)
        time.sleep(5)
        
        while True:
            print(f"--- Processando página {page_number} ---")
            
            # Tentar fechar popups
            try:
                cookie_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Aceitar') or contains(text(), 'OK')]")
                cookie_button.click()
            except:
                pass

            # Rolar a página
            for _ in range(3):
                driver.execute_script("window.scrollBy(0, 1000);")
                time.sleep(1)
            
            # Seletores de links
            selectors = ['a[itemprop="url"]', 'div[data-testid="card-container"] a', 'a[href*="/rooms/"]']
            links_found = []
            for selector in selectors:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                if elements: links_found.extend(elements)
            
            if not links_found:
                print(f"Nenhum link encontrado na página {page_number}.")
                break

            current_page_new_links = 0
            for link in links_found:
                try:
                    href = link.get_attribute('href')
                    if href and '/rooms/' in href:
                        # Mantém as datas na URL do link individual
                        clean_url = href.split('?')[0] + f"?check_in={CHECK_IN}&check_out={CHECK_OUT}"
                        if clean_url not in all_links:
                            all_links.add(clean_url)
                            current_page_new_links += 1
                except:
                    continue
            
            print(f"Novos links nesta página: {current_page_new_links}. Total: {len(all_links)}")

            # Paginação
            try:
                next_button = None
                next_selectors = ['a[aria-label="Próximo"]', 'a[aria-label="Next"]', '//a[contains(@href, "section_offset")]']
                for sel in next_selectors:
                    try:
                        if sel.startswith('//'): next_button = driver.find_element(By.XPATH, sel)
                        else: next_button = driver.find_element(By.CSS_SELECTOR, sel)
                        if next_button: break
                    except: continue

                if next_button:
                    next_url = next_button.get_attribute('href')
                    driver.get(next_url)
                    page_number += 1
                    time.sleep(5)
                else:
                    print("Fim das páginas de resultados.")
                    break
            except:
                break

    finally:
        driver.quit()
    
    return list(all_links)

if __name__ == "__main__":
    links = scrape_airbnb_links()
    
    output_file = "airbnb_links_with_dates.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(links, f, indent=4, ensure_ascii=False)
    
    print(f"\nExtração concluída para {CIDADE}-{ESTADO}!")
    print(f"Total de links únicos extraídos: {len(links)}")
    print(f"Resultados salvos em: {os.path.abspath(output_file)}")

Iniciando busca para: Conde, Paraiba
URL de busca: https://www.airbnb.com.br/s/Conde--Paraiba/homes?check_in=2026-03-10&check_out=2026-03-15
--- Processando página 1 ---
Novos links nesta página: 24. Total: 24
--- Processando página 2 ---
Novos links nesta página: 18. Total: 42
--- Processando página 3 ---
Novos links nesta página: 18. Total: 60
--- Processando página 4 ---
Novos links nesta página: 17. Total: 77
--- Processando página 5 ---
Novos links nesta página: 18. Total: 95
--- Processando página 6 ---
Novos links nesta página: 18. Total: 113
--- Processando página 7 ---
Novos links nesta página: 17. Total: 130
--- Processando página 8 ---
Novos links nesta página: 18. Total: 148
--- Processando página 9 ---
Novos links nesta página: 18. Total: 166
--- Processando página 10 ---
Novos links nesta página: 18. Total: 184
--- Processando página 11 ---
Novos links nesta página: 17. Total: 201
--- Processando página 12 ---
Novos links nesta página: 17. Total: 218
--- Processando págin

### Extrai dados dos anúncios

In [ ]:
import time
import json
import os
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from geopy.geocoders import Nominatim

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def classify_category(title):
    title = title.lower()
    categories = {
        "Flat/Studio": ["flat", "studio", "loft"],
        "Apartamento": ["apartamento", "apto", "condomínio", "condo"],
        "Casa de temporada": ["casa", "residência", "villa", "mansão", "sobrado"],
        "Chalé": ["chalé", "chale", "cabana", "bungalow", "bangalô"],
        "Hostel": ["hostel", "albergue"],
        "Pousada": ["pousada", "suíte", "quarto"],
        "Hotel": ["hotel"],
        "Resort": ["resort"],
        "Hospedagem domiciliar": ["quarto em", "homestay"]
    }
    for cat, keywords in categories.items():
        if any(kw in title for kw in keywords):
            return cat
    return "Casa de temporada"

def convert_rating(rating_5):
    try:
        if not rating_5: return "NSA"
        # Limpa a string para pegar apenas o número (ex: "4,85" ou "4.85")
        match = re.search(r'(\d+[\.,]\d+)', str(rating_5))
        if not match:
            match = re.search(r'(\d+)', str(rating_5))
        
        if not match: return "NSA"
        
        val = float(match.group(1).replace(',', '.'))
        if val <= 0 or val > 5.0: return "NSA"
        
        val_10 = val * 2
        if val_10 >= 9.0: label = "Excepcional"
        elif val_10 >= 8.5: label = "Fantástico"
        elif val_10 >= 8.0: label = "Muito Bom"
        elif val_10 >= 7.0: label = "Bom"
        else: label = "Regular"
        
        return f"{val_10:.1f} {label}".replace('.', ',')
    except:
        return "NSA"

def main():
    input_file = "airbnb_links_with_dates.json"
    output_file = "airbnb_padronizado_booking.csv"
    
    if not os.path.exists(input_file):
        print(f"Erro: {input_file} não encontrado.")
        return
    
    with open(input_file, "r") as f:
        room_links = json.load(f)

    driver = setup_driver()
    geolocator = Nominatim(user_agent="airbnb_booking_integrator_v5")
    dados_coletados = []

    print(f"Iniciando extração de {len(room_links)} anúncios com estratégias avançadas de avaliação...")

    for i, link in enumerate(room_links):
        try:
            print(f"[{i+1}/10] Processando: {link}")
            driver.get(link)
            time.sleep(15)
            
            html = driver.page_source
            
            # 1. Título
            try:
                titulo = driver.find_element(By.TAG_NAME, "h1").text
            except:
                titulo = "N/A"

            # 2. Avaliação - Estratégias Sugeridas
            rating = None
            
            # Estratégia 1: Localizar via SVG da Estrela
            try:
                # Procura o SVG que contém o path específico da estrela
                star_path = "m15.1 1.58-4.13 8.88-9.86 1.27a1 1 0 0 0-.54 1.74l7.3 6.57-1.97 9.85a1 1 0 0 0 1.48 1.06l8.62-5 8.63 5a1 1 0 0 0 1.48-1.06l-1.97-9.85 7.3-6.57a1 1 0 0 0-.55-1.73l-9.86-1.28-4.12-8.88a1 1 0 0 0-1.82 0z"
                xpath_svg = f"//*[local-name()='svg']//*[local-name()='path' and @d='{star_path}']/ancestor::*[local-name()='svg']/.."
                parent_element = driver.find_element(By.XPATH, xpath_svg)
                # A nota geralmente está no texto do elemento pai ou vizinho
                text_content = parent_element.text
                match = re.search(r'(\d+[\.,]\d+)', text_content)
                if match:
                    rating = match.group(1)
                    print(f"   > Nota encontrada via SVG: {rating}")
            except: pass

            # Estratégia 2: Âncora de texto "avaliaç"
            if not rating:
                try:
                    # Procura elementos que contenham "avaliaç" e tenta pegar o número anterior
                    elements = driver.find_elements(By.XPATH, "//*[contains(text(), 'avaliaç')]")
                    for el in elements:
                        parent_text = el.find_element(By.XPATH, "..").text
                        # Procura padrão: "4,85 (120 avaliações)" ou "4.85 · 120 avaliações"
                        match = re.search(r'(\d+[\.,]\d+)\s*.*avaliaç', parent_text)
                        if match:
                            rating = match.group(1)
                            print(f"   > Nota encontrada via âncora 'avaliaç': {rating}")
                            break
                except: pass

            # Fallback: Busca direta no HTML se as estratégias visuais falharem
            if not rating:
                match = re.search(r'([\d\.,]+)\s*·\s*\d+\s*avaliaç', html)
                if match:
                    rating = match.group(1)
                    print(f"   > Nota encontrada via Regex HTML: {rating}")

            classificacao = convert_rating(rating)

            # 3. Categoria
            categoria = classify_category(titulo)

            # 4. Capacidade
            hospedes = quartos = camas = 0
            try:
                overview = driver.find_element(By.CSS_SELECTOR, 'div[data-section-id="OVERVIEW_DEFAULT_V2"], div[data-testid="listing-details-not-standalone"]').text
                h_m = re.search(r'(\d+)\s+hóspede', overview)
                q_m = re.search(r'(\d+)\s+quarto', overview)
                c_m = re.search(r'(\d+)\s+cama', overview)
                if h_m: hospedes = int(h_m.group(1))
                if q_m: quartos = int(q_m.group(1))
                if c_m: camas = int(c_m.group(1))
            except: pass

            # 5. Preço
            preco = "N/A"
            try:
                price_text = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="book-it-default"]').text
                p_m = re.search(r'R\$\s*([\d\.,]+)', price_text)
                if p_m: preco = p_m.group(1).replace('.', '').replace(',', '')
            except: pass

            # 6. Endereço
            endereco = "N/A"
            try:
                coords = re.findall(r'"latitude":([\d\.\-]+),"longitude":([\d\.\-]+)', html)
                if coords:
                    loc = geolocator.reverse((coords[0][0], coords[0][1]), language='pt')
                    endereco = loc.address if loc else "N/A"
            except: pass

            # 7. Anfitrião
            anfitriao = "N/A"
            try:
                anfitriao = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="host-profile-container"] h2').text.replace("Anfitriã(o): ", "")
            except: pass

            dados_coletados.append({
                "Município (PB)": "Conde",
                "Categoria/Tipo": categoria,
                "Nome do empreendimento": titulo,
                "CNPJ": "",
                "Endereço da Hospedagem": endereco,
                "Imagens da Hospedagem e Localização no Mapa": link,
                "Classificação/Avaliação": classificacao,
                "Valor médio da diária (R$) por quarto duplo": preco,
                "Quantidade de quartos (UH) ": quartos,
                "Quantidade de camas (leitos)": camas,
                "Capacidade (Quantidade de pessoas)": hospedes,
                "Site/Rede sociais": "Airbnb",
                "Contatos (E-mail/Telefone)": anfitriao
            })

        except Exception as e:
            print(f"Erro no link {link}: {e}")

    df = pd.DataFrame(dados_coletados)
    cols = [
        "Município (PB)", "Categoria/Tipo", "Nome do empreendimento", "CNPJ",
        "Endereço da Hospedagem", "Imagens da Hospedagem e Localização no Mapa",
        "Classificação/Avaliação", "Valor médio da diária (R$) por quarto duplo",
        "Quantidade de quartos (UH) ", "Quantidade de camas (leitos)",
        "Capacidade (Quantidade de pessoas)", "Site/Rede sociais", "Contatos (E-mail/Telefone)"
    ]
    df = df[cols]
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"Concluído! Planilha salva em {output_file}")
    driver.quit()

if __name__ == "__main__":
    main()

Iniciando extração de 269 anúncios com estratégias avançadas de avaliação...
[1/10] Processando: https://www.airbnb.com.br/rooms/53311633?check_in=2026-03-10&check_out=2026-03-15
   > Nota encontrada via âncora 'avaliaç': 5,0
[2/10] Processando: https://www.airbnb.com.br/rooms/1422290680975031853?check_in=2026-03-10&check_out=2026-03-15
   > Nota encontrada via âncora 'avaliaç': 4,8
[3/10] Processando: https://www.airbnb.com.br/rooms/948365634189422947?check_in=2026-03-10&check_out=2026-03-15
   > Nota encontrada via âncora 'avaliaç': 4,9
[4/10] Processando: https://www.airbnb.com.br/rooms/1512452602317339244?check_in=2026-03-10&check_out=2026-03-15
   > Nota encontrada via âncora 'avaliaç': 4,89
[5/10] Processando: https://www.airbnb.com.br/rooms/1618162940651909335?check_in=2026-03-10&check_out=2026-03-15
[6/10] Processando: https://www.airbnb.com.br/rooms/1274728395035408092?check_in=2026-03-10&check_out=2026-03-15
   > Nota encontrada via âncora 'avaliaç': 5,0
[7/10] Processando: h